In [ ]:
import os 
os.environ["HF_HOME"] = "/your/path/hf_cache"
from datasets import load_dataset 

    

In [ ]:

eval_data = load_dataset('json', 
                         data_files="/your/path/visual7w/dataset.json", 
                         split="train")

In [ ]:
from PIL import Image
idx = 0
# print(eval_data[idx].keys()) 
# # dict_keys(['images', 'version', 'dataset'])
# print(eval_data[idx]["images"].keys()) 
# dict_keys(['filename', 'image_id', 'qa_pairs', 'split'])
for qa_pair in eval_data[idx]["images"]['qa_pairs']:
    print(qa_pair['question'], qa_pair['answer'])

In [ ]:
count = 0
daycount = 0
nightcount = 0
eveningcount = 0
afternooncount = 0
duskcount = 0
morningcount = 0
risecount = 0
for item in eval_data:
    if item["images"]["split"] != 'val': continue
    for qa_pair in item["images"]["qa_pairs"]:
        if "When" in qa_pair["question"] and "taken" in qa_pair["question"]:
            if "day" in qa_pair["answer"].lower(): daycount += 1
            elif "night" in qa_pair["answer"].lower(): nightcount += 1
            elif "after dark" in qa_pair["answer"].lower(): 
                nightcount += 1
                print(qa_pair["answer"])
            elif "evening" in qa_pair["answer"].lower(): eveningcount += 1
            elif "afternoon" in qa_pair["answer"].lower(): afternooncount += 1
            elif "dusk" in qa_pair["answer"].lower(): 
                duskcount += 1
                print(qa_pair["answer"])
            elif "morning" in qa_pair["answer"].lower(): morningcount += 1
            elif "sunrise" in qa_pair["answer"].lower(): 
                risecount += 1
        elif "When is it" in qa_pair["question"] or "When was it" in qa_pair["question"]:
            if "day" in qa_pair["answer"].lower(): daycount += 1
            elif "night" in qa_pair["answer"].lower(): nightcount += 1
            elif "after dark" in qa_pair["answer"].lower(): 
                nightcount += 1
                print(qa_pair["answer"])
            elif "evening" in qa_pair["answer"].lower(): eveningcount += 1
            elif "afternoon" in qa_pair["answer"].lower(): afternooncount += 1
            elif "dusk" in qa_pair["answer"].lower(): 
                duskcount += 1
                print(qa_pair["answer"])
            elif "morning" in qa_pair["answer"].lower(): morningcount += 1
            elif "sunrise" in qa_pair["answer"].lower(): 
                risecount += 1

print(daycount, nightcount, eveningcount, afternooncount, duskcount, morningcount, risecount)
            

In [ ]:
# Time: day, night/dark, evening, afternoon
categories = {"daytime":[], "night":[], "evening":[], "afternoon":[], "dusk": [], "morning": [], "sunrise": []}
for item in eval_data:
    if item["images"]["split"] != 'val': continue
    gt = None
    for qa_pair in item["images"]["qa_pairs"]:
        if "When" in qa_pair["question"] and "taken" in qa_pair["question"]:
            if "day" in qa_pair["answer"].lower(): gt = "daytime"
            elif "night" in qa_pair["answer"].lower(): gt = "night"
            elif "dark" in qa_pair["answer"].lower(): gt = "night"
            elif "evening" in qa_pair["answer"].lower(): gt = "evening"
            elif "afternoon" in qa_pair["answer"].lower(): gt = "afternoon"
            elif "dusk" in qa_pair["answer"].lower(): gt = "dusk"
            elif "morning" in qa_pair["answer"].lower(): gt = "morning"
            elif "sunrise" in qa_pair["answer"].lower(): gt = "sunrise"
        elif "When is it" in qa_pair["question"] or "When was it" in qa_pair["question"]:
            if "day" in qa_pair["answer"].lower(): gt = "daytime"
            elif "night" in qa_pair["answer"].lower(): gt = "night"
            elif "dark" in qa_pair["answer"].lower(): gt = "night"
            elif "evening" in qa_pair["answer"].lower(): gt = "evening"
            elif "afternoon" in qa_pair["answer"].lower(): gt = "afternoon"
            elif "dusk" in qa_pair["answer"].lower(): gt = "dusk"
            elif "morning" in qa_pair["answer"].lower(): gt = "morning"
            elif "sunrise" in qa_pair["answer"].lower(): gt = "sunrise"

    if gt != None: 
        categories[gt].append(item['images']['filename'])

In [ ]:
import random

new_data = []
# time_of_day
#   daytime
#     morning
#       sunrise
#     afternoon
#   night
#   evening = dusk
exclusive = {"daytime": ["night", "evening", "dusk"], 
             "morning": ["night", "evening", "dusk"], # Here, we exclude afternoon
             "sunrise": ["night", "evening", "dusk"],
             "afternoon": ["night", "evening", "dusk"],
             "night": ["daytime", "morning", "sunrise", "afternoon", "dusk", "evening"],
             "evening": ["daytime", "morning", "sunrise", "afternoon", "night"],
             "dusk": ["daytime", "morning", "sunrise", "afternoon", "night"]}
for label in categories:
    for data in categories[label]:
        new_item = {}
        new_item['qry_text'] = f"Find me an everyday image that is taken during the {label}.\n" # the scene of ...
        new_item['qry_img_path'] = ''
        new_item['tgt_text'] = "<|image_1|> Represent the given image."
        new_item['tgt_img_path'] = [data]
        all_paths = []
        for place in categories:
            if place in exclusive[label]:
                all_paths.extend(categories[place])
        new_item['tgt_img_path'].extend(random.sample(all_paths, 99))
        new_data.append(new_item)

In [ ]:
print(len(categories["dusk"]))
print(len(new_data))

In [ ]:
import json

with open('Visual7W_time_retrieval.json', 'w') as f:
    json.dump(new_data, f, indent=4)

In [ ]:
# Test whether we can load it using load_dataset
new_eval_data = load_dataset('json', 
                      data_files='Visual7W_time_retrieval.json',
                      split="train")

In [ ]:
idx = 1
print(new_eval_data[idx].keys())
print(new_eval_data[idx]['qry_text'])
img = Image.open("/your/path/visual7w/images/"+new_eval_data[idx]['tgt_img_path'][29]).convert("RGB")
img